# Limpieza de Datos

La limpieza de datos es el proceso de arreglar las anomalías un dataset, por ejemplo, excluyendo los datos que son incorrectos, corruptos, mal formateados, duplicados o incompletos; además, se busca la manera de reemplazar los datos perdidos por datos en concreto. Todo esto se hace con el fin de incrementar la consistencia y precisión de los análisis posteriores.

## ¿Qué tenemos que hacer?

- Cargar los datos
- Dar un vistazo a los datos, a los tipos de datos y a su dimensión para tener una visión global del _dataset_.
- Identificar datos:

    - Perdidos 
    - Repetitivos
    - Duplicados
    
- Corregir el _dataset_.


Para este _notebook_ vamos a partir cargando las librerías que necesitamos:

In [2]:
import pandas as pd
import numpy as np

Y ahora vamos a cargar nuestro _dataset_.

In [ ]:
df = pd.read_csv('datos_comunas_mod.csv')

Lamentablemente, **obtuvimos un error por el _encoding_ de los datos**. Pandas asumió que el archivo CSV estaba en formato UTF-8, pero se grabó desde Excel con otra codificación 'Latin1'. Este es un error común en Pandas, por lo que es bueno que sepas que existan distintas codificaciones a la hora de cargar un archivo.

In [5]:
df = pd.read_csv('datos_comunas_mod.csv',encoding='latin1')

Damos un primer vistazo a nuesto dataset. Para saber las dimensiones del Dataframe, ocupamos `shape`:

In [6]:
df.shape

(374, 4)

Con `head()`, tenemos una muestra de los datos presentes.

In [7]:
df.head()

,metros_plaza,metros_parque,personas,tiene arboles
0,58460.0,94824.0,271.0,Sí
1,130316.0,0.0,55.0,Sí
2,NaN,NaN,NaN,Sí
3,4050.0,0.0,8.0,Sí
4,2532.0,NaN,NaN,Sí


Y ahora con la función info, obtenemos el tipo de datos para cada columna además de la cantidad de datos no-nulo:

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   metros_plaza   353 non-null    float64
 1   metros_parque  345 non-null    float64
 2   personas       344 non-null    float64
 3   tiene arboles  374 non-null    object 
dtypes: float64(3), object(1)
memory usage: 11.8+ KB


## Identificando datos perdidos:

Con la función `isnull()`, podemos ver los nulos que hay en una determinada celda.

In [9]:
df.isnull()

,metros_plaza,metros_parque,personas,tiene arboles
0,False,False,False,False
1,False,False,False,False
2,True,True,True,False
3,False,False,False,False
4,False,True,True,False
...,...,...,...,...
369,False,False,False,False
370,False,False,False,False
371,False,False,False,False
372,False,False,False,False


Así, podemos sumar los nulos por columna:

In [10]:
df.isnull().sum()

metros_plaza     21
metros_parque    29
personas         30
tiene arboles     0
dtype: int64

¿Qué podemos hacer con los datos perdidos? En general, existen tres acciones principales (¡aunque este es un tópico bastante amplio por si solo!):

- Eliminar fila(s) 
- Eliminar columna
- Reemplazarlo con algún valor (conocido también como **imputar** el valor)

In [11]:
# Eliminar filas

# dropna() retorna un nuevo DataFrame
df_drop = df.dropna()
df_drop.head(20)

,metros_plaza,metros_parque,personas,tiene arboles
0,58460.0,94824.0,271.0,Sí
1,130316.0,0.0,55.0,Sí
3,4050.0,0.0,8.0,Sí
5,2300.0,0.0,17.0,Sí
6,12925.0,20749.0,18.0,Sí
7,409807.0,269533.0,383.0,Sí
9,8287.0,35608.0,24.0,Sí
11,283236.0,180901.0,159.0,Sí
12,2500.0,0.0,11.0,Sí
15,6410.0,0.0,23.0,Sí


También podemos eliminar la columna entera (que es una opción un poco radical):

In [13]:
# Eliminar columnas

df_del_columns = df.dropna(axis=1)
df_del_columns.head()

,tiene arboles
0,Sí
1,Sí
2,Sí
3,Sí
4,Sí


Como vemos, se nos pasó la mano, ya que la función `dropna(axis=1)` elimina cualquier columna que contenga un `NaN`. Además recordemos que el DataFrame original `df` **no ha cambiado**, porque las funciones retornan un nuevo objeto de tipo `DataFrame`.

In [15]:
df.head()

,metros_plaza,metros_parque,personas,tiene arboles
0,58460.0,94824.0,271.0,Sí
1,130316.0,0.0,55.0,Sí
2,NaN,NaN,NaN,Sí
3,4050.0,0.0,8.0,Sí
4,2532.0,NaN,NaN,Sí


Ahora vamos a reemplazar los valores nulos por un número en particular.

In [27]:
# En este caso vamos a reemplazar por un 0

df_filled = df.fillna(0)
df_filled.head()

,metros_plaza,metros_parque,personas,tiene arboles
0,58460.0,94824.0,271.0,Sí
1,130316.0,0.0,55.0,Sí
2,0.0,0.0,0.0,Sí
3,4050.0,0.0,8.0,Sí
4,2532.0,0.0,0.0,Sí


Otro método de imputación bastante utilizado es reemplazar los nulos por la mediana, la media o la moda de una determinada columna. Veamos ahora un ejemplo de cómo obtener el promedio, la mediana y la moda de una columna:

In [22]:
# Promedio
df['metros_plaza'].mean()

263440.7507082153

In [ ]:
# Promedio
df['metros_plaza'].median()

In [23]:
# Promedio
df['metros_plaza'].mode()

0    10000.0
dtype: float64

Como supondrás, en este caso al tener valores continuos bien dispersos, la moda no va a ser un buen método de reemplazo, pero cuando tenemos una variable categórica (por ejemplo, `color` que puede tomar valores `azul`, `rojo` o `verde`), es bastante más útil. En este caso vamos a escoger reemplazar por la mediana.

In [29]:
df_filled_mplaza

,metros_plaza
0,58460.0
1,130316.0
2,29585.0
3,4050.0
4,2532.0
...,...
369,168800.0
370,21282.0
371,21925.0
372,9500.0


In [30]:
median_mplaza = df['metros_plaza'].median() # obtengo mediana de metros de plaza.
median_mparque = df['metros_parque'].median() # obtengo mediana de metros de paque.
median_personas = df['personas'].median() # obtengo mediana de personas

print(f"La mediana metros plaza es: {median_mplaza}" )
print(f"La mediana metros parque es: {median_mparque}" )
print(f"La mediana personas es: {median_personas}" )

# Imputamos los NaN con los valores de las medianas.

# Ojo, aquí estamos modificando el dataframe original
# Esto es porque estamos haciendo una operación a nivel de columna
df['metros_plaza'].fillna(median_mplaza, inplace=True) 
df['metros_parque'].fillna(median_mparque, inplace=True)
df['personas'].fillna(median_personas, inplace=True)
df.head(20)

La mediana metros plaza es: 29585.0
La mediana metros parque es: 9400.0
La mediana personas es: 43.0


,metros_plaza,metros_parque,personas,tiene arboles
0,58460.0,94824.0,271.0,Sí
1,130316.0,0.0,55.0,Sí
2,29585.0,9400.0,43.0,Sí
3,4050.0,0.0,8.0,Sí
4,2532.0,9400.0,43.0,Sí
5,2300.0,0.0,17.0,Sí
6,12925.0,20749.0,18.0,Sí
7,409807.0,269533.0,383.0,Sí
8,11263.0,9400.0,28.0,Sí
9,8287.0,35608.0,24.0,Sí


Si quieres aprender más sobre métodos de imputación [puedes revisar el siguiente post](https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4). 

## Identificando datos duplicados

Los datos duplicados pueden ser un problema (como tener dos filas idénticas) son un problema, ya que puede indicar que un registro fue ingresado dos veces. Ahora vamos a enseñar cómo detectar filas duplicadas con Pandas.

In [35]:
# Cargamos el dataset de nuevo, porque lo habíamos modificado
df = pd.read_csv('datos_comunas_mod.csv',encoding='latin1')
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
369     True
370     True
371     True
372     True
373     True
Length: 374, dtype: bool

Como vemos, generamos una lista con las posiciones duplicadas (ojo, la primera aparición de una fila no es marcada como duplicada). Usamos el objeto anterior como filtro para ver las filas duplicadas.

In [38]:
df[df.duplicated()]

,metros_plaza,metros_parque,personas,tiene arboles
10,NaN,NaN,NaN,Sí
77,NaN,NaN,NaN,Sí
119,NaN,NaN,NaN,Sí
183,NaN,NaN,NaN,Sí
220,NaN,NaN,NaN,Sí
228,NaN,NaN,NaN,Sí
231,NaN,NaN,NaN,Sí
247,NaN,NaN,NaN,Sí
311,NaN,NaN,NaN,Sí
333,NaN,NaN,NaN,Sí


Para eliminar datos duplicados hacemos lo siguiente:

In [39]:
# Elimanos datos duplicados:

# Como supondrás, inplace=True nos hace modificar el DataFrame original
df.drop_duplicates(inplace=True)

# Vamos a checkear que en efecto, ya no hay ningún duplicado
df[df.duplicated()]

,metros_plaza,metros_parque,personas,tiene arboles


También podemos identificar datos repetitivos con el comando `unique`, que nos indica el número de valores únicos por columna:

In [40]:
df.nunique()

metros_plaza     312
metros_parque    187
personas         145
tiene arboles      1
dtype: int64

Si nos damos cuenta que hay una columna que siempre tiene el mismo valor, quizás nos gustaría eliminarla. Para eliminar columnas en Pandas hacemos lo siguiente:

In [41]:
# Eliminamos datos repetitivos:
df.drop('tiene arboles', axis=1, inplace=True)

In [42]:
df.head()

,metros_plaza,metros_parque,personas
0,58460.0,94824.0,271.0
1,130316.0,0.0,55.0
2,NaN,NaN,NaN
3,4050.0,0.0,8.0
4,2532.0,NaN,NaN


Como vemos, eliminamos la columna `tiene arboles`.